In [2]:
import os
import pandas as pd
from pingouin import intraclass_corr

In [3]:
folderA_path = "[0 data] scores/Rater 1"
folderB_path = "[0 data] scores/Rater 2"

# Accumulators
icc_scores = {"Engagement": []}
specificity_scores = {}
specificity_scores_by_study = {}

# List all files in both folders
filesA = sorted(os.listdir(folderA_path))
filesB = sorted(os.listdir(folderB_path))

filesA = [item for item in filesA if item != ".DS_Store"]
filesB = [item for item in filesB if item != ".DS_Store"]

# Process each pair of files
i = 0
for fileA, fileB in zip(filesA, filesB):
    i += 1
    if fileA != fileB:
        print(f"File mismatch: {fileA} and {fileB}")
        continue

    if fileA == "ignore":
        continue

    pathA = os.path.join(folderA_path, fileA)
    pathB = os.path.join(folderB_path, fileB)
    
    dfA = pd.read_excel(pathA)
    dfB = pd.read_excel(pathB)

    specificity_cols = [col for col in dfA.columns if "specificity" in col.lower()]

    dfA = dfA[["Engagement"] + specificity_cols]
    dfB = dfB[["Engagement"] + specificity_cols]

    # Align the datasets based on row indices
    dfA = dfA.dropna()
    dfB = dfB.dropna()
    dfA, dfB = dfA.align(dfB, join="inner", axis=0)

    if dfB.empty:
        continue

    def normalize(df):
        return (df - df.min()) / (df.max() - df.min())

    dfA = normalize(dfA)
    dfB = normalize(dfB)

    for var in ["Engagement"]:
        if var in dfA.columns and var in dfB.columns:
            # ICC can only be calculated if both raters have at least 5 ratings.
            # All studies have more than 5 interventions, so this should be fine.
            if len(dfB[var]) < 5:
                continue

            iccdfA = pd.DataFrame({var: dfA[var], "intervention": dfA.index, "rater": "Rater 1"})
            iccdfB = pd.DataFrame({var: dfB[var], "intervention": dfB.index, "rater": "Rater 2"})

            combined_data = pd.concat([iccdfA, iccdfB], ignore_index=True)

            icc = intraclass_corr(data=combined_data, targets='intervention', raters='rater', ratings=var)
            icc.set_index("Type")

            icc_score = icc.loc[icc["Type"] == "ICC2k", "ICC"].values[0]

            icc_scores[var].append(icc_score)

    for col in specificity_cols:
        if col in dfA.columns and col in dfB.columns:
            if len(dfB[col]) < 5:
                continue

            iccdfA = pd.DataFrame({col: dfA[col], "intervention": dfA.index, "rater": "Rater 1"})
            iccdfB = pd.DataFrame({col: dfB[col], "intervention": dfB.index, "rater": "Rater 2"})
            combined_data = pd.concat([iccdfA, iccdfB], ignore_index=True)

            icc = intraclass_corr(data=combined_data, targets='intervention', raters='rater', ratings=col)
            icc.set_index("Type")
            
            icc_score = icc.loc[icc["Type"] == "ICC2k", "ICC"].values[0]
            specificity_scores[col] = icc_score
            specificity_scores_by_study[fileA] = specificity_scores_by_study.get(fileA, [])
            specificity_scores_by_study[fileA].append(icc_score)

# Averages
average_icc = {key: sum(values) / len(values) if values else None for key, values in icc_scores.items()}
clean_specificity_scores = {k: v for k, v in specificity_scores.items() if not pd.isna(v)}
average_specificity_icc = sum(clean_specificity_scores.values()) / len(clean_specificity_scores)
average_specificity_icc_by_study = {k: sum(v) / len(v) for k, v in specificity_scores_by_study.items() if v}

print("Average ICC for specificity columns:", average_specificity_icc)
print("Average ICC for specificity columns by study:", average_specificity_icc_by_study)


/Users/yashua/Documents/vlasceanu/intervention features/.venv/lib/python3.11/site-packages/pingouin/reliability.py:307: RuntimeWarning: invalid value encountered in scalar divide
  msw = (aov.at[1, "SS"] + aov.at[2, "SS"]) / (aov.at[1, "DF"] + aov.at[2, "DF"])
/Users/yashua/Documents/vlasceanu/intervention features/.venv/lib/python3.11/site-packages/pingouin/reliability.py:307: RuntimeWarning: invalid value encountered in scalar divide
  msw = (aov.at[1, "SS"] + aov.at[2, "SS"]) / (aov.at[1, "DF"] + aov.at[2, "DF"])
/Users/yashua/Documents/vlasceanu/intervention features/.venv/lib/python3.11/site-packages/pingouin/reliability.py:307: RuntimeWarning: invalid value encountered in scalar divide
  msw = (aov.at[1, "SS"] + aov.at[2, "SS"]) / (aov.at[1, "DF"] + aov.at[2, "DF"])


Average ICC for specificity columns: 0.6851692357523312
Average ICC for specificity columns by study: {'Bruneau et al..xlsx': np.float64(0.9347417979802669), 'Dellavigna & Pope, 2018.xlsx': np.float64(0.5142857142857137), 'Duckworth et al., 2025.xlsx': np.float64(0.8120372581800811), 'Fazio et al_.xlsx': np.float64(0.3574326282237033), 'Goldwert et al_.xlsx': np.float64(0.5480805512238914), 'Lai et al.xlsx': np.float64(0.893385214007782), 'Mason et al., 2025.xlsx': np.float64(0.7321234119782214), 'Milkman et al., 2021.xlsx': np.float64(0.4964336661911554), 'Milkman et al., 2022.xlsx': np.float64(0.05357890330682289), 'Milkman et al., 2024.xlsx': np.float64(0.9387755102040817), 'Sinclair et al. 2025.xlsx': np.float64(0.581842841940557), 'Tappin et al. 2023 imm.xlsx': np.float64(nan), 'Tappin et al. 2023 ubi.xlsx': np.float64(nan), 'Vlasceanu et al., 2024.xlsx': np.float64(0.7460492505394393), 'Voelkel et al., 2023.xlsx': np.float64(0.7929312417386192), 'Zickfeld et al., 2024.xlsx': np.f